In [2]:
pip install sqlalchemy

Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func
import datetime as dt
from config import db_password

In [4]:
# Change the path to use a different one
file_path = Path("./resources/listings_cleaned2.csv")
df = pd.read_csv(file_path,low_memory=False)
df.head()

,id,host_since,host_response_time,host_response_rate,host_is_superhost,host_listings_count,host_total_listings_count,host_identity_verified,neighbourhood_cleansed,property_type,...,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,host_days_active,time_since_first_review,time_since_last_review
0,958,2008-07-31,within a few hours,100%,1,1,1,1,Western Addition,Apartment,...,9.0,STR-0001256,1,1,1,0,0,3958.0,4+ years,0-2 weeks
1,7918,2009-06-17,within a few hours,100%,0,10,10,1,Haight Ashbury,Apartment,...,8.0,NaN,0,9,0,9,0,3637.0,4+ years,6-12 months
2,8142,2009-06-17,within a few hours,100%,0,10,10,1,Haight Ashbury,Apartment,...,9.0,NaN,0,9,0,9,0,3637.0,4+ years,2-6 months
3,8339,2009-07-02,within an hour,100%,0,2,2,1,Western Addition,House,...,9.0,STR-0000264,0,2,2,0,0,3622.0,4+ years,2-6 months
4,8739,2009-01-27,within an hour,100%,1,2,2,0,Mission,Apartment,...,10.0,STR-0000028,1,2,0,2,0,3778.0,4+ years,0-2 weeks


In [5]:
df.columns.tolist()

['id',
 'host_since',
 'host_response_time',
 'host_response_rate',
 'host_is_superhost',
 'host_listings_count',
 'host_total_listings_count',
 'host_identity_verified',
 'neighbourhood_cleansed',
 'property_type',
 'room_type',
 'accommodates',
 'bathrooms',
 'bedrooms',
 'beds',
 'price',
 'minimum_nights',
 'maximum_nights',
 'availability_30',
 'availability_60',
 'availability_90',
 'availability_365',
 'number_of_reviews',
 'first_review',
 'review_scores_rating',
 'review_scores_accuracy',
 'review_scores_cleanliness',
 'review_scores_checkin',
 'review_scores_communication',
 'review_scores_location',
 'review_scores_value',
 'license',
 'instant_bookable',
 'calculated_host_listings_count',
 'calculated_host_listings_count_entire_homes',
 'calculated_host_listings_count_private_rooms',
 'calculated_host_listings_count_shared_rooms',
 'host_days_active',
 'time_since_first_review',
 'time_since_last_review']

In [6]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df2 = df.copy()
cols = ['host_since']
df2[cols] = df2[cols].apply(LabelEncoder().fit_transform)
df2.head()

,id,host_since,host_response_time,host_response_rate,host_is_superhost,host_listings_count,host_total_listings_count,host_identity_verified,neighbourhood_cleansed,property_type,...,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,host_days_active,time_since_first_review,time_since_last_review
0,958,2,within a few hours,100%,1,1,1,1,Western Addition,Apartment,...,9.0,STR-0001256,1,1,1,0,0,3958.0,4+ years,0-2 weeks
1,7918,14,within a few hours,100%,0,10,10,1,Haight Ashbury,Apartment,...,8.0,NaN,0,9,0,9,0,3637.0,4+ years,6-12 months
2,8142,14,within a few hours,100%,0,10,10,1,Haight Ashbury,Apartment,...,9.0,NaN,0,9,0,9,0,3637.0,4+ years,2-6 months
3,8339,17,within an hour,100%,0,2,2,1,Western Addition,House,...,9.0,STR-0000264,0,2,2,0,0,3622.0,4+ years,2-6 months
4,8739,6,within an hour,100%,1,2,2,0,Mission,Apartment,...,10.0,STR-0000028,1,2,0,2,0,3778.0,4+ years,0-2 weeks


In [9]:
df_encoded = pd.get_dummies(df, columns=["host_response_rate", "host_total_listings_count", "property_type","price"])
df_encoded.head()

,id,host_since,host_response_time,host_is_superhost,host_listings_count,host_identity_verified,neighbourhood_cleansed,room_type,accommodates,bathrooms,...,price_$980.00,price_$984.00,price_$986.00,price_$989.00,price_$99.00,price_$990.00,price_$994.00,price_$995.00,price_$998.00,price_$999.00
0,958,2008-07-31,within a few hours,1,1,1,Western Addition,Entire home/apt,3,1,...,0,0,0,0,0,0,0,0,0,0
1,7918,2009-06-17,within a few hours,0,10,1,Haight Ashbury,Private room,2,4,...,0,0,0,0,0,0,0,0,0,0
2,8142,2009-06-17,within a few hours,0,10,1,Haight Ashbury,Private room,2,4,...,0,0,0,0,0,0,0,0,0,0
3,8339,2009-07-02,within an hour,0,2,1,Western Addition,Entire home/apt,5,1.5,...,0,0,0,0,0,0,0,0,0,0
4,8739,2009-01-27,within an hour,1,2,0,Mission,Private room,3,1,...,0,0,0,0,0,0,0,0,0,0


In [10]:
df_encoded.describe()

,id,host_is_superhost,host_listings_count,host_identity_verified,accommodates,bedrooms,beds,minimum_nights,maximum_nights,availability_30,...,price_$980.00,price_$984.00,price_$986.00,price_$989.00,price_$99.00,price_$990.00,price_$994.00,price_$995.00,price_$998.00,price_$999.00
count,1.261600e+04,12616.000000,12616.000000,12616.000000,12616.000000,11677.000000,12314.000000,1.261600e+04,1.261600e+04,12616.000000,...,12616.000000,12616.000000,12616.000000,12616.000000,12616.000000,12616.000000,12616.000000,12616.000000,12616.000000,12616.000000
mean,2.256558e+07,0.447210,72.997543,0.733830,3.177473,1.444121,1.801039,7.945891e+03,8.432656e+03,8.180327,...,0.000079,0.000079,0.000079,0.000079,0.015060,0.000396,0.000079,0.000555,0.000238,0.003408
std,1.507179e+07,0.497225,287.527885,0.441972,1.888229,0.902529,1.199249,8.903055e+05,8.903018e+05,10.500955,...,0.008903,0.008903,0.008903,0.008903,0.121797,0.019905,0.008903,0.023550,0.015419,0.058284
min,9.580000e+02,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000e+00,1.000000e+00,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,8.923590e+06,0.000000,1.000000,0.000000,2.000000,1.000000,1.000000,2.000000e+00,2.900000e+01,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2.220792e+07,0.000000,2.000000,1.000000,2.000000,1.000000,1.000000,5.000000e+00,1.800000e+02,2.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,3.329019e+07,1.000000,10.000000,1.000000,4.000000,2.000000,2.000000,3.000000e+01,1.125000e+03,14.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,5.310006e+07,1.000000,1987.000000,1.000000,16.000000,14.000000,14.000000,1.000000e+08,1.000000e+08,30.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
